In [42]:
# Imports
import pandas as pd
import numpy as np
import gzip

In [43]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)





# Preprocessing

## AKAs Cleaning

- Keep only US movies
- Replace "\N" with np.nan

In [44]:
# Filter US movies
akas = akas[akas['region'] == 'US']
print(akas['region'].value_counts())

# Replace "\N" with np.nan
akas.replace("\\N", np.nan, inplace=True)

US    1460475
Name: region, dtype: int64


In [45]:
# Check for NaNs
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1456353
types               477606
attributes         1413133
isOriginalTitle       1342
dtype: int64

## Titles Cleaning
- Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)
- Replace "\N" with np.nan
- Eliminate movies that are null for runtimeMinutes
- Eliminate movies that are null for genre
- keep only titleType==Movie
- Convert the startYear column to float data type.
- Filter the dataframe using startYear. Keep years between 2000-2021 (Including 2000 and 2021)
- Eliminate movies that include "Documentary" in the genre (see tip below).

In [46]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers = basics['tconst'].isin(akas['titleId'])

In [47]:
# Filter Basics based on the Akas dataframe
basics = basics[basics['tconst'].isin(akas['titleId'])]

# Replace "\N" with np.NaN
basics.replace("\\N", np.nan, inplace=True)

# Confirm that nans exist
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            0
originalTitle           0
isAdult                 0
startYear           99233
endYear           1335556
runtimeMinutes     505573
genres              28562
dtype: int64

In [48]:
# Remove movies that are null for runtimeMinutes and genre
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

# Keep only titleType == 'movie'
basics = basics[basics['titleType'] == 'movie']

# Convert startYear to float data type
basics['startYear'] = basics['startYear'].astype(float)

# Filter the dataframe using startYear
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2021)]

# Remove movies that have the "Documentary" genere
basics = basics[~basics['genres'].str.contains('Documentary')]

## Ratings Cleaning

In [49]:
# Keep only movies that were included in the final title basics dataframe
ratings = ratings[ratings['tconst'].isin(basics['tconst'])]

# Replace "\N" with np.nan
ratings.replace("\\N", np.nan, inplace=True)

In [50]:
# Check for nans in Ratings
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

# Export dataframes to files

In [51]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [52]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [53]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [54]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()



,tconst,averageRating,numVotes
0,tt0035423,6.4,87333
1,tt0062336,6.4,179
2,tt0068865,5.4,75
3,tt0069049,6.7,7793
4,tt0088751,5.2,338


In [55]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [56]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()



,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [57]:
display(basics.info())
display(akas.info())
display(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82005 entries, 0 to 82004
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82005 non-null  object 
 1   titleType       82005 non-null  object 
 2   primaryTitle    82005 non-null  object 
 3   originalTitle   82005 non-null  object 
 4   isAdult         82005 non-null  int64  
 5   startYear       82005 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  82005 non-null  int64  
 8   genres          82005 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460475 entries, 0 to 1460474
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1460475 non-null  object 
 1   ordering         1460475 non-null  int64  
 2   title            1460475 non-null  object 
 3   region           1460475 non-null  object 
 4   language         4122 non-null     object 
 5   types            982869 non-null   object 
 6   attributes       47342 non-null    object 
 7   isOriginalTitle  1459133 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 89.1+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68127 entries, 0 to 68126
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         68127 non-null  object 
 1   averageRating  68127 non-null  float64
 2   numVotes       68127 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB


None